# Elementy Inteligencji Obliczeniowej - Sieci Neuronowe


---

**Prowadzący:** Jakub Bednarek<br>
**Kontakt:** jakub.bednarek@put.poznan.pl<br>
**Materiały:** [Strona WWW](http://jakub.bednarek.pracownik.put.poznan.pl)

---

## Uwaga

* **Aby wykonać polecenia należy najpierw przejść do trybu 'playground'. File -> Open in Playground Mode**
* Nowe funkcje Colab pozwalają na autouzupełnianie oraz czytanie dokumentacji

## Cel ćwiczeń:
- zapoznanie się z rekurencyjnymi sieciami neuronowymi,
- stworzenie modelu sieci z warstwami rekurencyjnymi dla zbioru danych MNIST,
- stworzenie własnych implementacji warstwami neuronowych

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D, LSTM, LSTMCell, SimpleRNNCell
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta, RMSprop
from tensorflow.python.keras import backend as K

In [4]:
(x_tr, y_tr), (x_te, y_te) = mnist.load_data()
x_tr = x_tr.astype('float32') # shape: 60000, 28, 28
x_te = x_te.astype('float32') # shape: 10000, 28, 28
x_tr /= 255  # normalizacja wartości do przedziału [0, 1]
x_te /= 255
y_tr = to_categorical(y_tr, 10)  # zamiana etykiety na one-hot encoding; np. 2 -> [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y_te = to_categorical(y_te, 10)

11493376/11490434 [==============================] - 0s 0us/step


## Sieci rekurencyjne
http://colah.github.io/posts/2015-08-Understanding-LSTMs/

https://www.tensorflow.org/guide/keras/rnn

https://www.tensorflow.org/guide/function

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/

Przykładowy model z warstwą rekurencyjną dla danych MNIST:

In [5]:
class RecurrentModel(Model):
    def __init__(self, num_classes=10):
        super().__init__(name='my_model')
        self.num_classes = num_classes
        self.lstm_1 = LSTM(128, activation='relu')
        self.dense_1 = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.lstm_1(inputs)
        return self.dense_1(x)

model = RecurrentModel(num_classes=10)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_tr, y_tr, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 52s 27ms/step - loss: 1.1463 - accuracy: 0.6448
Epoch 2/2
1875/1875 [==============================] - 51s 27ms/step - loss: 0.1589 - accuracy: 0.9538


### Zadanie 1
Rozszerz model z powyższego przykładu o kolejną warstwę rekurencyjną przed gęstą warstwą wyjściową.

Standardowe sieci neuronowe generują jeden wynik na podstawie jednego inputu, natomiast sieci rekurencyjne przetwarzają dane sekwencyjnie, w każdym kroku łącząc wynik poprzedniego przetwarzania i aktualnego wejścia. Dlatego domyślnym wejściem sieci neuronowej jest tensor 3-wymiarowy ([batch_size,sequence_size,sample_size]).
Domyślnie warstwy rekurencyjne w Kerasie zwracają tylko wyniki przetwarzania ostatniego
kroku (otrzymują tensor 3-wymiarowy, zwracają tensor 2-wymiarowy). Jeśli chcesz zwrócić sekwencje wyników wszystkich kroków przetwarzania dla warstwy rekurencyjnej, musisz ustawić parametr return_sequences=True.


In [7]:
class Model1(Model):
    def __init__(self, num_classes):
        super().__init__(name='Model1')
        self.num_classes = num_classes
        self.lstm_1 = LSTM(128, activation='relu', return_sequences=True)
        self.lstm_2 = LSTM(128, activation='relu')
        self.dense_1 = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.lstm_1(inputs)
        x = self.lstm_2(x)
        return self.dense_1(x)

model = Model1(num_classes=10)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_tr, y_tr, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 112s 59ms/step - loss: 1.1394 - accuracy: 0.6693
Epoch 2/2
1875/1875 [==============================] - 112s 60ms/step - loss: 0.1497 - accuracy: 0.9582


### Zadanie 2 
Wykorzystując model z przykładu, napisz sieć rekurencyjną przy użyciu SimpleRNNCell.

Cell implementuje tylko operacje wykonywane przez warstwę
rekurencyjną dla jednego kroku. Warstwy rekurencyjne w każdym kroku
łączą wynik operacji poprzedniego kroku i aktualny input.
Wykorzystaj pętle for do wielokrotnego wywołania komórki SimpleRNNCell (liczba kroków to liczba elementów w sekwencji). Aby wywołać SimpleRNNCell dla pojedynczego wejścia i stanu należy użyć jej metody ```call``` analogicznie jak w przypadku własnych modeli (tzn. ```my_model(input)```). 



Wywołanie zainicjalizowanej komórki rekurencyjnej wymaga podania aktualnego inputu i **listy macierzy** (w dokumentacji jest błąd, że ma to być macierz) stanów ukrytych poprzedniego kroku (SimpleRNNCell ma jeden stan, LSTMCell w następnym zadaniu ma dwa stany).

Trzeba zainicjalizować ukryty stan warstwy wartościami początkowymi (można wykorzystać rozkład normalny - tf.random.normal).

In [8]:
class Model2(Model):
    def __init__(self, num_classes):
        super().__init__(name='Model2')
        self.num_classes = num_classes
        self.cell = SimpleRNNCell(128, activation='relu')
        self.dense_1 = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        h = tf.random.normal([inputs.shape[0], self.cell.units])
        for i in range(inputs.shape[1]):
          x, h = self.cell(inputs[:,i,:], h)
        return self.dense_1(x)

model = Model2(num_classes=10)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_tr, y_tr, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 13s 6ms/step - loss: 0.7982 - accuracy: 0.7297
Epoch 2/2
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2206 - accuracy: 0.9343


### Zadanie 3
Zamień komórkę rekurencyjną z poprzedniego zadania na LSTMCell (LSTMCell ma dwa stany ukryte).

In [37]:
class Model3(Model):
    def __init__(self, num_classes):
        super().__init__(name='Model3')
        self.num_classes = num_classes
        self.cell = LSTMCell(128, activation='relu')
        self.dense_1 = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        shape = (inputs.shape[0], self.cell.units)
        h = [tf.random.normal(shape), tf.random.normal(shape)]
        for i in range(inputs.shape[1]):
          x, h = self.cell(inputs[:,i,:], h)
        return self.dense_1(x)

model = Model3(num_classes=10)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_tr, y_tr, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 48s 24ms/step - loss: 1.2776 - accuracy: 0.6218
Epoch 2/2
1875/1875 [==============================] - 44s 23ms/step - loss: 0.1583 - accuracy: 0.9549


### Zadanie 4
Wykorzystując model z poprzedniego zadania, stwórz model sieci
neuronowej z własną implementacją prostej warstwy rekurencyjnej.
- w call zamień self.lstm_cell_layer(x) na wywołanie własnej metody np. self.cell(x)
- w konstruktorze modelu usuń inicjalizację komórki LSTM i zastąp ją inicjalizacją warstw potrzebnych do stworzenia własnej komórki rekurencyjnej,
- stwórz metodę cell() wykonującą operacje warstwy rekurencyjnej,
- prosta warstwa rekurencyjna konkatenuje poprzedni wyniki i aktualny input, a następnie przepuszcza ten połączony tensor przez warstwę gęstą (Dense).

In [38]:
class Model4(Model):
    def __init__(self, num_classes):
        super().__init__(name='Model4')
        self.num_classes = num_classes
        self._cell = Dense(128, activation='relu')
        self.dense_1 = Dense(num_classes, activation='softmax')

    def cell(self, x, h):
        return self._cell(K.concatenate([x, h]))

    def call(self, inputs):
        h = tf.random.normal((inputs.shape[0], self._cell.units))
        for i in range(inputs.shape[1]):
          h = self.cell(inputs[:,i,:], h)
        return self.dense_1(h)

model = Model4(num_classes=10)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_tr, y_tr, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 14s 7ms/step - loss: 0.8494 - accuracy: 0.7116
Epoch 2/2
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2451 - accuracy: 0.9280


### Zadanie 5

Na podstawie modelu z poprzedniego zadania stwórz model z własną implementacją warstwy LSTM. Dokładny i zrozumiały opis działania warstwy LSTM znajduje się na [stronie](http://colah.github.io/posts/2015-08-Understanding-LSTMs/).

In [45]:
class Model5(Model):
    def __init__(self, num_classes):
        super().__init__(name='Model5')
        self.num_classes = num_classes
        self.units = 128
        self.dense_i = Dense(self.units, activation='sigmoid')
        self.dense_f = Dense(self.units, activation='sigmoid')
        self.dense_o = Dense(self.units, activation='sigmoid')
        self.dense_c = Dense(self.units, activation='tanh')
        self.dense_1 = Dense(num_classes, activation='softmax')

    def cell(self, x, h):
        c_last, h_last = h
        hx = K.concatenate([h_last, x])
        c_next = self.dense_f(hx)*c_last + self.dense_i(hx)*self.dense_c(hx)
        h_next = self.dense_o(hx)*K.tanh(c_next)
        return h_next, (c_next, h_next)

    def call(self, inputs):
        shape = (inputs.shape[0], self.units)
        h = [tf.random.normal(shape), tf.random.normal(shape)]
        for i in range(inputs.shape[1]):
          x, h = self.cell(inputs[:,i,:], h)
        return self.dense_1(x)

model = Model5(num_classes=10)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_tr, y_tr, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 40s 19ms/step - loss: 0.6215 - accuracy: 0.7974
Epoch 2/2
1875/1875 [==============================] - 36s 19ms/step - loss: 0.1355 - accuracy: 0.9617
